In [ ]:
%%capture
!pip install unsloth vllm
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install git+https://github.com/huggingface/transformers.git

In [ ]:
import os
# os.environ['TRITON_DISABLE_LINE_INFO'] = '1' # Optional, tested with and without
os.environ['TRITON_JIT_DISABLE_OPT'] = '1' # Likely the most critical change

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from unsloth import FastLanguageModel

model_name = "unsloth/DeepSeek-R1-0528-Qwen3-8B-unsloth-bnb-4bit"  # or 3.2, 70B, Instruct, etc.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=1024,
    load_in_4bit=True,  # use QLoRA
    dtype=None,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 07-03 04:05:00 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 07-03 04:05:01 [__init__.py:239] Automatically detected platform cuda.


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


==((====))==  Unsloth 2025.6.12: Fast Qwen3 patching. Transformers: 4.54.0.dev0. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}
Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

generation_config.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes


In [ ]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0.0,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    bias="none",
    use_gradient_checkpointing = False,
    use_rslora=False,
)
tokenizer = get_chat_template(tokenizer, chat_template="chatml")

Unsloth 2025.6.12 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.
Unsloth: Will map <|im_end|> to EOS = <｜end▁of▁sentence｜>.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [ ]:
import json
from datasets import Dataset

# Load your JSON dataset
with open("combined_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Convert to chat format
def format_example(entry):
    user_prompt = (
        f"Question: {entry['question']}\n"
        f"Rubric: {entry['rubric']}\n"
        f"Ideal Answer: {entry['ideal_answer']}\n"
        f"Student Answer: {entry['student_answer']}\n"
        f"Strategy: {entry['strategy']}"
    )
    assistant_response = f"Score: {entry['score']}\nReview: {entry['cot_review']}"
    return {
        "user": user_prompt,
        "assistant": assistant_response
    }

# Apply transformation
chat_data = [format_example(entry) for entry in data]
ds = Dataset.from_list(chat_data)


In [ ]:
ds[0]

{'user': "Question: Choose three principles from the list provided and explain how they can be applied to improve the agile software development process. Provide examples for each principle.\nRubric: Demonstration of 'Learn From History' principle (2 marks)\nApplication of 'Less Code Is Better Code' principle (2 marks)\nExplanation of 'Make The System Usable' principle (2 marks)\nExamples provided for each principle (2 marks)\nWriting style and organization (2 marks)\nIdeal Answer: The response demonstrates a strong understanding of the 'Learn From History' principle, highlighting the importance of building on past experiences to make informed decisions in the agile development process. The answer also illustrates the value of 'Less Code Is Better Code', providing examples of how concise and clear code can lead to a more maintainable and efficient system. Furthermore, the response emphasizes the significance of 'Make The System Usable', discussing the importance of user-friendly interf

In [ ]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template

# Get tokenizer (assume already loaded)
tokenizer = get_chat_template(tokenizer, chat_template="chatml")

# Apply formatting to dataset
ds = ds.map(lambda example: {
    "text": tokenizer.apply_chat_template(
        [{"role": "user", "content": example["user"]},
         {"role": "assistant", "content": example["assistant"]}],
        tokenize=False
    )
})


Unsloth: Will map <|im_end|> to EOS = <|im_end|>.


Map:   0%|          | 0/911 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    fp16 = False,
    num_train_epochs=1,
    learning_rate=3e-5,
    save_total_limit=2,
    logging_steps=10,
    max_steps = 200,
      # optional
)


def formatting_func(examples):
    return {"text": examples["text"]}

trainer = SFTTrainer(
    model=model,
    train_dataset=ds, # Use the processed dataset `ds`
    args=training_args,
    tokenizer=tokenizer,
    formatting_func=formatting_func, # Add the formatting function
)


Unsloth: Tokenizing ["text"]:   0%|          | 0/911 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 911 | Num Epochs = 2 | Total steps = 200
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 43,646,976 of 8,234,382,336 (0.53% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gaurishk75 (gaurishk75-vellore-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.489900
20,1.329100
30,1.231500
40,1.139800
50,1.055300
60,0.977500
70,0.960700
80,0.916800
90,0.901900
100,0.870900


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.
Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


TrainOutput(global_step=200, training_loss=0.9445944142341614, metrics={'train_runtime': 3879.4293, 'train_samples_per_second': 0.412, 'train_steps_per_second': 0.052, 'total_flos': 5.569044580429824e+16, 'train_loss': 0.9445944142341614})

In [ ]:
model.save_pretrained("Deepseek-grader")
tokenizer.save_pretrained("Deepseek-grader")


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


('Deepseek-grader/tokenizer_config.json',
 'Deepseek-grader/special_tokens_map.json',
 'Deepseek-grader/chat_template.jinja',
 'Deepseek-grader/tokenizer.json')

In [ ]:
chat = [
    {"role": "user", "content": f"""
"Question": "What is the primary objective of requirements modeling, and how does it contribute to the software development process?",
    "Rubric": "A clear description of the primary objective of requirements modeling, including its role in describing customer requirements (3 marks)\nAn explanation of how requirements modeling establishes a basis for software design, including the importance of bridging the gap between system-level representation and software design (4 marks)\nA discussion of the role of requirements modeling in defining a set of requirements that can be validated once the software is built, including the significance of validation in the software development process (3 marks)",
    "Ideal Answer": "The primary objective of requirements modeling is to create a variety of representations that describe what the customer requires, establishing a basis for the creation of a software design, and defining a set of requirements that can be validated once the software is built. Requirements modeling plays a crucial role in bridging the gap between a system-level representation that describes overall system and business functionality and a software design. By doing so, it enables the development of a software design that meets the customer's requirements and ensures that the software functions as intended. Furthermore, requirements modeling defines a set of requirements that can be validated once the software is built, allowing for the verification of the software's functionality and performance. This validation is essential in ensuring that the software meets the customer's needs and is free from defects, ultimately contributing to the success of the software development process.",
    "Student Answer": "To be honest, I'm not entirely sure about the primary objective of requirements modeling, but from what I've gathered, it's basically about creating a bunch of diagrams and documents that describe what the customer wants. I think it's supposed to help us understand the customer's requirements, but I'm not really clear on how it does that. \n\nI've heard that requirements modeling is important for software design, but I don't really get how it bridges the gap between system-level representation and software design. I mean, isn't software design just about writing code? I'm not sure how requirements modeling fits into that. Maybe it's like, a way to make sure we're building the right thing, but I'm not really sure.\n\nAs for validation, I think requirements modeling is supposed to help us define a set of requirements that we can check against once the software is built. But, to be honest, I'm not really sure how that works or why it's important. I know validation is a thing, but I don't really understand its significance in the software development process. Can we come back to this later? I feel like I'm missing something. \n\nOh, and I think requirements modeling might also have something to do with, like, making sure the software is usable and stuff. But, yeah, that's about it. I'm pretty lost on this topic, sorry.",
    "Strategy": "clarification"
     Please evaluate the student's answer by assigning a score from 0 to 10 and providing a concise review.
Return only this format:
Score: <score from 0 to 10>
Review: <your feedback>
"""}

]

response = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(response, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Qwen3ForCausalLM has no `_prepare_4d_causal_attention_mask_with_cache_position` method defined in its base modeling class. Compiled forward passes will be sub-optimal. If you're writing code, see Llama for an example implementation. If you're a user, please report this issue on GitHub.


user

"Question": "What is the primary objective of requirements modeling, and how does it contribute to the software development process?",
    "Rubric": "A clear description of the primary objective of requirements modeling, including its role in describing customer requirements (3 marks)
An explanation of how requirements modeling establishes a basis for software design, including the importance of bridging the gap between system-level representation and software design (4 marks)
A discussion of the role of requirements modeling in defining a set of requirements that can be validated once the software is built, including the significance of validation in the software development process (3 marks)",
    "Ideal Answer": "The primary objective of requirements modeling is to create a variety of representations that describe what the customer requires, establishing a basis for the creation of a software design, and defining a set of requirements that can be validated once the software is

In [ ]:
chat = [
    {"role": "user", "content": f"""
Question": "What are the limitations of using use cases as a modeling tool in software engineering, and how can UML models supplement these limitations?",
    "Rubric": "Description of the limitations of use cases, including situations where they may not be sufficient, such as safety critical systems (3 marks)\nExplanation of the role of scenario-based modeling in software engineering and its appropriateness for a majority of situations (4 marks)\nDiscussion of how UML models can provide substantial benefits as a modeling tool when developed properly, including their ability to supplement use cases (3 marks)",
    "Ideal Answer": "Use cases have limitations as a modeling tool in software engineering, particularly in situations that require significant detail and precision, such as safety critical systems. In these cases, a use case may not be sufficient to capture the complexity of the system. However, scenario-based modeling is appropriate for a significant majority of situations that software engineers encounter. When developed properly, use cases can provide substantial benefits as a modeling tool, including the ability to capture functional requirements and identify key actors and systems. UML models can supplement the limitations of use cases by providing a more detailed and formal representation of the system, allowing for the capture of non-functional requirements and the analysis of system behavior. By combining use cases with UML models, software engineers can create a more comprehensive and accurate model of the system, ultimately leading to better design and development outcomes.",
    "Student Answer": "To be honest, I'm not entirely sure about the limitations of using use cases as a modeling tool in software engineering, but I'll try to break it down. \n\nFrom what I understand, use cases are basically a way to describe how users interact with a system, and they're really useful for understanding the functional requirements of a project. However, I think one of the limitations of use cases is that they might not be detailed enough for complex systems, like safety-critical systems. For instance, in systems where safety is a top priority, such as in healthcare or aerospace, use cases alone might not be sufficient to capture all the possible scenarios and edge cases that could lead to safety risks. \n\nIn terms of scenario-based modeling, I believe it's a pretty common approach in software engineering, and it's suitable for most situations. Scenario-based modeling involves creating scenarios that describe how the system should behave under different conditions, and it can help identify potential issues and requirements. I think this approach is useful because it allows developers to think about how the system will be used in real-life situations, which can help them design a more user-friendly and effective system. However, I'm not entirely sure how scenario-based modeling relates to use cases, but I think they might be connected in some way.\n\nRegarding UML models, I'm a bit fuzzy on how they can supplement the limitations of use cases. From what I've learned, UML (Unified Modeling Language) is a standard language for software modeling, and it provides a set of tools for creating diagrams that describe the structure and behavior of a system. I think UML models can be useful for creating a more detailed and formal representation of a system, which can help supplement the limitations of use cases. For example, UML models can provide a more detailed description of the system's architecture and behavior, which can help identify potential issues and requirements that might not be captured by use cases alone. However, I'm not entirely sure how to develop UML models properly, or how they can provide substantial benefits as a modeling tool. \n\nIn summary, I think use cases have limitations, especially in complex systems, and scenario-based modeling is a useful approach for understanding how a system will be used. UML models can also be useful for supplementing the limitations of use cases, but I'm not entirely sure how to develop them properly or how they can provide substantial benefits.",
    "Strategy": "clarification"
     Please evaluate the student's answer by assigning a score from 0 to 10 and providing a concise review.
Return only this format:
Score: <score from 0 to 10>
Review: <your feedback>
"""}

]

response = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(response, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

user

Question": "What are the limitations of using use cases as a modeling tool in software engineering, and how can UML models supplement these limitations?",
    "Rubric": "Description of the limitations of use cases, including situations where they may not be sufficient, such as safety critical systems (3 marks)
Explanation of the role of scenario-based modeling in software engineering and its appropriateness for a majority of situations (4 marks)
Discussion of how UML models can provide substantial benefits as a modeling tool when developed properly, including their ability to supplement use cases (3 marks)",
    "Ideal Answer": "Use cases have limitations as a modeling tool in software engineering, particularly in situations that require significant detail and precision, such as safety critical systems. In these cases, a use case may not be sufficient to capture the complexity of the system. However, scenario-based modeling is appropriate for a significant majority of situations 

In [ ]:
chat = [
    {"role": "user", "content": f"""
"Question": "What are the limitations of using use cases as a modeling tool in software engineering, and how can UML models supplement these limitations?",
    "Rubric": "Clear explanation of the limitations of use cases, including scenarios where they may not be sufficient (e.g., safety critical systems) (3 marks)\nDescription of the benefits of using use cases as a modeling tool when developed properly (2 marks)\nDiscussion of how UML models can supplement use cases, including the types of situations where UML models are appropriate (5 marks)",
    "Ideal Answer": "Use cases have limitations as a modeling tool in software engineering, particularly in situations that require significant detail and precision, such as safety critical systems. In these scenarios, a use case may not provide enough information to ensure the system meets the necessary requirements. However, when developed properly, use cases can provide substantial benefits as a modeling tool, allowing software engineers to capture the functional requirements of a system. UML models can supplement use cases by providing a more detailed and formal representation of the system, making them appropriate for a significant majority of situations encountered in software engineering. By using UML models in conjunction with use cases, software engineers can create a more comprehensive model of the system, ensuring that all requirements are met and the system is designed to be safe, reliable, and efficient.",
    "Student Answer": "In software engineering, use cases are a fundamental modeling tool used to capture the functional requirements of a system from the user's perspective. However, they have several limitations that can impact their effectiveness in certain scenarios.\n\nOne of the primary limitations of use cases is that they may not be sufficient for modeling complex systems, particularly those that are safety-critical or have strict regulatory requirements. For instance, in systems like aircraft control or medical devices, the consequences of failure can be catastrophic, and use cases alone may not provide the necessary level of detail and rigor to ensure reliability and safety. Additionally, use cases can become cumbersome and difficult to manage when dealing with large, complex systems, making it challenging to identify and prioritize requirements.\n\nDespite these limitations, use cases can be a powerful modeling tool when developed properly. They provide a clear and concise way to communicate system functionality to stakeholders, including users, developers, and project managers. When use cases are well-structured and thoroughly analyzed, they can help ensure that the system meets the user's needs and expectations, reducing the risk of misunderstandings and miscommunication.\n\nTo supplement the limitations of use cases, UML (Unified Modeling Language) models can be used to provide a more comprehensive and detailed representation of the system. UML models, such as class diagrams, sequence diagrams, and state machine diagrams, can help to elaborate on the use cases by providing a more formal and precise description of the system's structure and behavior. For example, in safety-critical systems, UML models can be used to model the system's fault tolerance and error handling mechanisms, providing a more detailed and rigorous analysis of the system's behavior under various scenarios.\n\nUML models are particularly useful in situations where use cases are insufficient, such as:\n\n* Modeling complex system interactions and behaviors\n* Analyzing system performance and scalability\n* Designing and implementing software architectures\n* Integrating multiple systems and subsystems\n* Modeling real-time systems and embedded systems\n\nIn these situations, UML models can provide a more detailed and formal representation of the system, helping to identify and mitigate potential risks and errors. By combining use cases with UML models, software engineers can create a more comprehensive and accurate model of the system, ensuring that it meets the user's needs and expectations while also satisfying the necessary safety, security, and regulatory requirements. Overall, the use of UML models can help to supplement the limitations of use cases, providing a more robust and reliable approach to software engineering.",
    "Strategy": "target-guided"
     Please evaluate the student's answer by assigning a score from 0 to 10 and providing a concise review.
Return only this format:
Score: <score from 0 to 10>
Review: <your feedback>
"""}

]

response = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(response, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

user

"Question": "What are the limitations of using use cases as a modeling tool in software engineering, and how can UML models supplement these limitations?",
    "Rubric": "Clear explanation of the limitations of use cases, including scenarios where they may not be sufficient (e.g., safety critical systems) (3 marks)
Description of the benefits of using use cases as a modeling tool when developed properly (2 marks)
Discussion of how UML models can supplement use cases, including the types of situations where UML models are appropriate (5 marks)",
    "Ideal Answer": "Use cases have limitations as a modeling tool in software engineering, particularly in situations that require significant detail and precision, such as safety critical systems. In these scenarios, a use case may not provide enough information to ensure the system meets the necessary requirements. However, when developed properly, use cases can provide substantial benefits as a modeling tool, allowing software engineer

In [ ]:
chat = [
    {"role": "user", "content": f"""
"Question": "What are the key tasks involved in requirements engineering, and how do they contribute to the overall success of a software development project?",
    "Rubric": "Identification of the primary tasks in requirements engineering, such as requirements gathering and management (3 marks)\nExplanation of the importance of each task in ensuring the quality and accuracy of software requirements (4 marks)\nDiscussion of the practical techniques and methods used to perform these tasks, including those proposed by Wiegers, Hull, Bray, and Arlow (3 marks)",
    "Ideal Answer": "Requirements engineering involves several key tasks that are crucial to the success of a software development project. The primary tasks include requirements gathering, which involves eliciting and collecting requirements from stakeholders, and requirements management, which involves analyzing, documenting, and maintaining these requirements. These tasks are essential in ensuring the quality and accuracy of software requirements, as they help to identify inconsistencies, ambiguities, and conflicts. Practical techniques for requirements gathering and management include those proposed by Wiegers, such as interviewing stakeholders and using use cases to capture functional requirements. Hull, Bray, and Arlow also provide methods for performing these tasks, including the use of requirements specification templates and formal analysis techniques. By using these techniques and methods, software developers can ensure that their requirements are complete, consistent, and unambiguous, which in turn contributes to the overall success of the project.",
    "Student Answer": "To be honest, I'm not entirely sure about the specific tasks involved in requirements engineering, but I'll try my best to provide an overview. From what I understand, requirements engineering is like, a crucial part of software development that helps ensure the final product meets the user's needs.\n\nThe primary tasks in requirements engineering seem to involve... um, let's see... requirements gathering, which is like, collecting all the necessary information from stakeholders, and requirements management, which is about, you know, keeping track of all the requirements and making sure they're consistent. I think there might be some other tasks too, like requirements analysis and validation, but I'm not really sure how they fit in.\n\nAs for the importance of each task, I believe that requirements gathering is essential because it helps developers understand what the users want. If you don't gather the right requirements, you might end up building something that nobody needs. Requirements management is also important because it ensures that all the requirements are, like, consistent and don't contradict each other. But, to be honest, I'm not really sure how these tasks contribute to the overall success of a software development project.\n\nIn terms of practical techniques and methods, I've heard of some guys like Wiegers, Hull, Bray, and Arlow who proposed some stuff, but I'm not really familiar with their work. I think they might have written some books or articles on requirements engineering, but I haven't really read them. Maybe they talked about, like, use cases and user stories, or something? I'm not really sure.\n\nOverall, I think requirements engineering is important, but I'm not really sure about the specifics. I hope this answer gives you some idea of what I'm thinking, but please keep in mind that I'm not really confident in my understanding of this topic.",
    "Strategy": "clarification"
     Please evaluate the student's answer by assigning a score from 0 to 10 and providing a concise review.
Return only this format:
Score: <score from 0 to 10>
Review: <your feedback>
"""}#2

]

response = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(response, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

user

"Question": "What are the key tasks involved in requirements engineering, and how do they contribute to the overall success of a software development project?",
    "Rubric": "Identification of the primary tasks in requirements engineering, such as requirements gathering and management (3 marks)
Explanation of the importance of each task in ensuring the quality and accuracy of software requirements (4 marks)
Discussion of the practical techniques and methods used to perform these tasks, including those proposed by Wiegers, Hull, Bray, and Arlow (3 marks)",
    "Ideal Answer": "Requirements engineering involves several key tasks that are crucial to the success of a software development project. The primary tasks include requirements gathering, which involves eliciting and collecting requirements from stakeholders, and requirements management, which involves analyzing, documenting, and maintaining these requirements. These tasks are essential in ensuring the quality and accuracy of

In [ ]:
chat = [
    {"role": "user", "content": f"""
"Question": "What are the key tasks involved in requirements engineering, and how do they contribute to the overall success of a software development project?",
    "Rubric": "Identification of the primary tasks in requirements engineering, such as requirements gathering and management (3 marks)\nExplanation of the importance of each task in ensuring the quality and accuracy of software requirements (4 marks)\nDiscussion of the practical techniques and methods used to perform these tasks, including those proposed by Wiegers, Hull, Bray, and Arlow (3 marks)",
    "Ideal Answer": "Requirements engineering involves several key tasks that are crucial to the success of a software development project. The primary tasks include requirements gathering, which involves eliciting and collecting requirements from stakeholders, and requirements management, which involves analyzing, documenting, and maintaining these requirements. These tasks are essential in ensuring the quality and accuracy of software requirements, as they help to identify inconsistencies, ambiguities, and conflicts. Practical techniques for requirements gathering and management include those proposed by Wiegers, such as interviewing stakeholders and using use cases to capture functional requirements. Hull, Bray, and Arlow also provide methods for performing these tasks, including the use of requirements specification templates and formal analysis techniques. By using these techniques and methods, software developers can ensure that their requirements are complete, consistent, and unambiguous, which in turn contributes to the overall success of the project.",
    "Student Answer": "Requirements engineering is a crucial part of software development that involves a bunch of tasks to ensure the software meets the needs of its users. The primary tasks in requirements engineering are requirements gathering, management, and also testing. \n\nRequirements gathering is when you collect all the requirements from the stakeholders, which is pretty important because you need to know what the users want. If you don't gather the requirements correctly, you might end up building something that nobody wants to use. Requirements management is also important because it involves keeping track of all the requirements and making sure they don't conflict with each other. Testing is necessary to ensure the software works as expected.\n\nEach of these tasks is important in its own way. Requirements gathering is essential because it sets the foundation for the whole project. If the requirements are not gathered correctly, the whole project could be a failure. Requirements management is also crucial because it helps to prevent scope creep and ensures that the requirements are testable. Testing is vital because it helps to ensure the software meets the requirements and works as expected.\n\nThere are many techniques and methods that can be used to perform these tasks. For example, Wiegers proposes the use of templates to gather requirements, while Hull and Bray suggest using agile methodologies to manage requirements. Arlow also proposes the use of use cases to gather requirements. These techniques and methods can be really helpful in ensuring the quality and accuracy of software requirements. However, the best approach often depends on the specific project and its requirements. \n\nOverall, requirements engineering is a critical part of software development, and performing these tasks correctly can make all the difference in the success of a project. By following the right techniques and methods, software developers can ensure that their software meets the needs of its users and is of high quality.",
    "Strategy": "target-guided"
     Please evaluate the student's answer by assigning a score from 0 to 10 and providing a concise review.
Return only this format:
Score: <score from 0 to 10>
Review: <your feedback>
"""}

]#7

response = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(response, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

user

"Question": "What are the key tasks involved in requirements engineering, and how do they contribute to the overall success of a software development project?",
    "Rubric": "Identification of the primary tasks in requirements engineering, such as requirements gathering and management (3 marks)
Explanation of the importance of each task in ensuring the quality and accuracy of software requirements (4 marks)
Discussion of the practical techniques and methods used to perform these tasks, including those proposed by Wiegers, Hull, Bray, and Arlow (3 marks)",
    "Ideal Answer": "Requirements engineering involves several key tasks that are crucial to the success of a software development project. The primary tasks include requirements gathering, which involves eliciting and collecting requirements from stakeholders, and requirements management, which involves analyzing, documenting, and maintaining these requirements. These tasks are essential in ensuring the quality and accuracy of

In [ ]:
trainer.save_model("/content/drive/MyDrive/fine-tuned-Deepseek")

Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


In [ ]:
chat = [
    {"role": "user", "content": f"""
"Question": "What is the primary objective of requirements modeling, and how does it contribute to the software development process?",
    "Rubric": "A clear description of the primary objective of requirements modeling, including its role in describing customer requirements (3 marks)\nAn explanation of how requirements modeling establishes a basis for software design, including the importance of bridging the gap between system-level representation and software design (4 marks)\nA discussion of the role of requirements modeling in defining a set of requirements that can be validated once the software is built, including the significance of validation in the software development process (3 marks)",
    "Ideal Answer": "The primary objective of requirements modeling is to create a variety of representations that describe what the customer requires, establishing a basis for the creation of a software design, and defining a set of requirements that can be validated once the software is built. Requirements modeling plays a crucial role in bridging the gap between a system-level representation that describes overall system and business functionality and a software design. By doing so, it enables the development of a software design that meets the customer's requirements and ensures that the software functions as intended. Furthermore, requirements modeling defines a set of requirements that can be validated once the software is built, allowing for the verification of the software's functionality and performance. This validation is essential in ensuring that the software meets the customer's needs and is free from defects, ultimately contributing to the success of the software development process.",
    "Student Answer": "To be honest, I'm not entirely sure about the primary objective of requirements modeling, but from what I've gathered, it's basically about creating a bunch of diagrams and documents that describe what the customer wants. I think it's supposed to help us understand the customer's requirements, but I'm not really clear on how it does that. \n\nI've heard that requirements modeling is important for software design, but I don't really get how it bridges the gap between system-level representation and software design. I mean, isn't software design just about writing code? I'm not sure how requirements modeling fits into that. Maybe it's like, a way to make sure we're building the right thing, but I'm not really sure.\n\nAs for validation, I think requirements modeling is supposed to help us define a set of requirements that we can check against once the software is built. But, to be honest, I'm not really sure how that works or why it's important. I know validation is a thing, but I don't really understand its significance in the software development process. Can we come back to this later? I feel like I'm missing something. \n\nOh, and I think requirements modeling might also have something to do with, like, making sure the software is usable and stuff. But, yeah, that's about it. I'm pretty lost on this topic, sorry.",
    "Strategy": "clarification"
     Please evaluate the student's answer by assigning a score from 0 to 10 and providing a concise review.
Return only this format:
Score: <score from 0 to 10>
Review: <your feedback>
"""}

]
response = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(response, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

user

"Question": "What is the primary objective of requirements modeling, and how does it contribute to the software development process?",
    "Rubric": "A clear description of the primary objective of requirements modeling, including its role in describing customer requirements (3 marks)
An explanation of how requirements modeling establishes a basis for software design, including the importance of bridging the gap between system-level representation and software design (4 marks)
A discussion of the role of requirements modeling in defining a set of requirements that can be validated once the software is built, including the significance of validation in the software development process (3 marks)",
    "Ideal Answer": "The primary objective of requirements modeling is to create a variety of representations that describe what the customer requires, establishing a basis for the creation of a software design, and defining a set of requirements that can be validated once the software is

In [ ]:
chat = [
    {"role": "user", "content": f"""
"Question": "What is dynamic programming and why is it useful in algorithm design?",
"Rubric": "A clear definition of dynamic programming, including its use of overlapping subproblems and optimal substructure (3 marks)\nAn explanation of how it differs from recursion and its efficiency benefits (4 marks)\nA discussion of its usefulness in solving problems like Fibonacci, knapsack, and matrix chain multiplication (3 marks)",
"Ideal Answer": "Dynamic programming (DP) is an algorithmic technique used to solve problems by breaking them down into simpler subproblems and storing their results to avoid redundant computations. It is applicable to problems with overlapping subproblems and optimal substructure. Unlike plain recursion, DP uses memoization or tabulation to improve efficiency, reducing time complexity significantly. For example, calculating Fibonacci numbers using recursion is exponential, while using DP makes it linear. DP is particularly useful in solving optimization problems like the 0/1 Knapsack, matrix chain multiplication, and longest common subsequence.",
"Student Answer": "Dynamic programming is like recursion but faster. It stores values so we don't repeat work. It’s used in problems like knapsack, I think. I'm not sure what optimal substructure means, but it helps in improving performance. I always get confused between top-down and bottom-up, but I know it’s useful.",
"Strategy": "clarification"
Please evaluate the student's answer by assigning a score from 0 to 10 and providing a concise review.
Return only this format:
Score: <score from 0 to 10>
Review: <your feedback>
"""}
]
response = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(response, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

user

"Question": "What is dynamic programming and why is it useful in algorithm design?",
"Rubric": "A clear definition of dynamic programming, including its use of overlapping subproblems and optimal substructure (3 marks)
An explanation of how it differs from recursion and its efficiency benefits (4 marks)
A discussion of its usefulness in solving problems like Fibonacci, knapsack, and matrix chain multiplication (3 marks)",
"Ideal Answer": "Dynamic programming (DP) is an algorithmic technique used to solve problems by breaking them down into simpler subproblems and storing their results to avoid redundant computations. It is applicable to problems with overlapping subproblems and optimal substructure. Unlike plain recursion, DP uses memoization or tabulation to improve efficiency, reducing time complexity significantly. For example, calculating Fibonacci numbers using recursion is exponential, while using DP makes it linear. DP is particularly useful in solving optimization problem

In [ ]:
chat = [
    {"role": "user", "content": f"""
"Question": "What are the key tasks involved in requirements engineering, and how do they contribute to the overall success of a software development project?",
    "Rubric": "Identification of the primary tasks in requirements engineering, such as requirements gathering and management (3 marks)\nExplanation of the importance of each task in ensuring the quality and accuracy of software requirements (4 marks)\nDiscussion of the practical techniques and methods used to perform these tasks, including those proposed by Wiegers, Hull, Bray, and Arlow (3 marks)",
    "Ideal Answer": "Requirements engineering involves several key tasks that are crucial to the success of a software development project. The primary tasks include requirements gathering, which involves eliciting and collecting requirements from stakeholders, and requirements management, which involves analyzing, documenting, and maintaining these requirements. These tasks are essential in ensuring the quality and accuracy of software requirements, as they help to identify inconsistencies, ambiguities, and conflicts. Practical techniques for requirements gathering and management include those proposed by Wiegers, such as interviewing stakeholders and using use cases to capture functional requirements. Hull, Bray, and Arlow also provide methods for performing these tasks, including the use of requirements specification templates and formal analysis techniques. By using these techniques and methods, software developers can ensure that their requirements are complete, consistent, and unambiguous, which in turn contributes to the overall success of the project.",
    "Student Answer": "Requirements engineering involves understanding what the customer wants and making sure the software matches those needs. The main tasks include gathering the requirements and checking them regularly. Gathering is when we talk to users and figure out their expectations. We can do interviews or surveys. Managing the requirements means keeping everything organized and making sure new ideas don’t mess up the old ones. This helps avoid confusion.

Some people like Wiegers have suggested that it's good to use diagrams and documentation to make things clear. Hull and Bray also talked about using agile to keep changing the requirements if needed. These things help make sure we don’t miss anything important.

These steps are important because if we skip any, we might build the wrong thing. So following these tasks helps in making sure the software works and satisfies users.",
    "Strategy": "target-guided"
     Please evaluate the student's answer by assigning a score from 0 to 10 and providing a concise review.
Return only this format:
Score: <score from 0 to 10>
Review: <your feedback>
"""}

]#7

response = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(response, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

user

"Question": "What are the key tasks involved in requirements engineering, and how do they contribute to the overall success of a software development project?",
    "Rubric": "Identification of the primary tasks in requirements engineering, such as requirements gathering and management (3 marks)
Explanation of the importance of each task in ensuring the quality and accuracy of software requirements (4 marks)
Discussion of the practical techniques and methods used to perform these tasks, including those proposed by Wiegers, Hull, Bray, and Arlow (3 marks)",
    "Ideal Answer": "Requirements engineering involves several key tasks that are crucial to the success of a software development project. The primary tasks include requirements gathering, which involves eliciting and collecting requirements from stakeholders, and requirements management, which involves analyzing, documenting, and maintaining these requirements. These tasks are essential in ensuring the quality and accuracy of

In [ ]:
chat = [
    {"role": "user", "content": f"""
"Question": "What are the key tasks involved in requirements engineering, and how do they contribute to the overall success of a software development project?",
    "Rubric": "Identification of the primary tasks in requirements engineering, such as requirements gathering and management (3 marks)\nExplanation of the importance of each task in ensuring the quality and accuracy of software requirements (4 marks)\nDiscussion of the practical techniques and methods used to perform these tasks, including those proposed by Wiegers, Hull, Bray, and Arlow (3 marks)",
    "Ideal Answer": "Requirements engineering involves several key tasks that are crucial to the success of a software development project. The primary tasks include requirements gathering, which involves eliciting and collecting requirements from stakeholders, and requirements management, which involves analyzing, documenting, and maintaining these requirements. These tasks are essential in ensuring the quality and accuracy of software requirements, as they help to identify inconsistencies, ambiguities, and conflicts. Practical techniques for requirements gathering and management include those proposed by Wiegers, such as interviewing stakeholders and using use cases to capture functional requirements. Hull, Bray, and Arlow also provide methods for performing these tasks, including the use of requirements specification templates and formal analysis techniques. By using these techniques and methods, software developers can ensure that their requirements are complete, consistent, and unambiguous, which in turn contributes to the overall success of the project.",
    "Student Answer": "Requirements engineering is mostly about finding out what the users want and writing it down. It's kind of like asking them what they need and then just building it. Once you have the requirements, the developers just follow them. You also test the software at the end to make sure it's working.

I think Wiegers said something about writing things clearly. I’m not sure what Hull or Arlow said, but they probably had some methods too. Anyway, it's just important to get the requirements early so you don’t mess up later.",
    "Strategy": "target-guided"
     Please evaluate the student's answer by assigning a score from 0 to 10 and providing a concise review.
Return only this format:
Score: <score from 0 to 10>
Review: <your feedback>
"""}

]#7

response = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(response, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

user

"Question": "What are the key tasks involved in requirements engineering, and how do they contribute to the overall success of a software development project?",
    "Rubric": "Identification of the primary tasks in requirements engineering, such as requirements gathering and management (3 marks)
Explanation of the importance of each task in ensuring the quality and accuracy of software requirements (4 marks)
Discussion of the practical techniques and methods used to perform these tasks, including those proposed by Wiegers, Hull, Bray, and Arlow (3 marks)",
    "Ideal Answer": "Requirements engineering involves several key tasks that are crucial to the success of a software development project. The primary tasks include requirements gathering, which involves eliciting and collecting requirements from stakeholders, and requirements management, which involves analyzing, documenting, and maintaining these requirements. These tasks are essential in ensuring the quality and accuracy of